In [10]:
"""
    처음 구성된 dataset을 우리가 사용할 목적에 맞게 specific query에 대해서만 data를 만들어주는 notebook
    speicific query

"""

import json

split = 'train'
data_path = './data/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
n_meetings = len(data)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 35 meetings in the test set.


In [11]:
from nltk import word_tokenize
# tokneize a sent
# 문장을 단어 단위로 쪼개는 느낌
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [12]:
# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [13]:
# process data for BART
# the input of the model here is the entire content of the meeting
from tqdm import tqdm
specific_data = []

for i in tqdm(range(len(data))):
    # get meeting content
    passage= []
    # print(f'length of data: {len(data[i])}')
    for k in range(len(data[i]['meeting_transcripts'])):
        cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ' said ' # 화자
        # cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': ' # 화자
        cur_turn = clean_data(cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])) # 말하는 말까지 포함해서 append
        passage.append(cur_turn)
    passage = ' \n '.join(passage)
    # for j in range(len(data[i]['general_query_list'])):
    #     cur = {}
    #     query = tokenize(data[i]['general_query_list'][j]['query'])
    #     cur['src'] = clean_data('<s> ' + query + ' </s> ' + src + ' </s>')
    #     target = tokenize(data[i]['general_query_list'][j]['answer'])
    #     cur['tgt'] = target
    #     bart_data.append(cur)

    for j in range(len(data[i]['specific_query_list'])):
        cur = dict()
        query = tokenize(data[i]['specific_query_list'][j]['query'])
        target = tokenize(data[i]['specific_query_list'][j]['answer']) 
        rel_span= [[int(start), int(end)] for start, end in data[i]['specific_query_list'][j]['relevant_text_span']]
        
        if len(rel_span) >1: print(rel_span)
         
        relevant_span= ''
        for h, (start, end) in enumerate(rel_span):
            # print(start, end)
            tmp_span= []
            for k in range(start, end+1):
                rel_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ' said ' # 화자
                # rel_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': ' # 화자
                rel_turn = clean_data(rel_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])) # 말하는 말까지 포함해서 append
                tmp_span.append(rel_turn)
            relevant_span+= ' \n '.join(tmp_span)
            if len(rel_span) > 1 and h != len(rel_span)-1: # rel span이 2개 이상일 때 + 마지막 span에는 추가해주지 않는다.
                relevant_span+= ' \n '
                # relevant_span+= ' \n\n '

        cur['query'] = clean_data(query)
        cur['passage']= clean_data(passage)
        cur['target'] = target
        cur['relevant_span']= clean_data(relevant_span)
        cur['document_id']= i
        cur['relevant_idx']= rel_span
        specific_data.append(cur)
        
print('Total {} query-summary pairs in the {} set'.format(len(specific_data), split))
# print(specific_data[2])
with open('./data/specific_' + split + '.jsonl', 'w') as f:
    for i in range(len(specific_data)):
        print(json.dumps(specific_data[i]), file=f)

  6%|▌         | 2/35 [00:00<00:06,  5.13it/s]

[[48, 76], [153, 389]]
[[77, 152], [390, 430]]
[[73, 88], [121, 175]]


 20%|██        | 7/35 [00:01<00:04,  6.06it/s]

[[43, 97], [297, 361]]
[[98, 193], [208, 251]]
[[199, 207], [252, 296], [362, 381]]
[[264, 283], [368, 379]]


 23%|██▎       | 8/35 [00:01<00:04,  5.48it/s]

...

[[119, 119], [171, 188]]


 94%|█████████▍| 33/35 [00:05<00:00,  4.95it/s]

[[30, 63], [267, 329]]
[[64, 64], [197, 202]]
[[66, 193], [203, 207]]


100%|██████████| 35/35 [00:06<00:00,  5.58it/s]

[[107, 118], [137, 186]]
[[119, 136], [278, 297]]
Total 244 query-summary pairs in the test set
